---
title: 极客时间-现代C++30讲  01丨堆、栈、RAII：C++里该如何管理资源？
tags: 小书匠,C++,极客时间-现代C++30讲,raii,heap,free store,new,malloc,delete,free
grammar_cjkRuby: true
---

[toc]

# 01丨堆、栈、RAII：C++里该如何管理资源？

## Two ways to manage objects: Heap and Stack

C++ provides two ways to manage objects: heap and stack.

### Stack

The terminology `stack` in C++ memory management is kind of similar to this terminology in data structures, which holds a feature of FILO(first in last out).

The objects constructed on stack will be destructed when it is out of scope of its lifetime. The most important thing is that this behavior still happens even an error is thrown during the execution of the function. The behavior is named `stack unwinding`.

Thanks to stack unwinding, stack may be the most natural way to manage memory in C++.

## Heap and free store

Heap is totally different from the heap in data structure, indicating the memory allocated dynamically. In C/C++, `malloc` is used to create a memory space on heap. 

Unfortunately, C/C++ doesn't provide automatic memory recycling mechanism, we need to free the allocated space using `free`.

Another similar concept is `free store`, specifying the memory on heap allocated managed using `new` and `delete`. Because `new` and `delete` are implemented using `malloc` and `free`, thus `free store` is a special kind of `heap`. Here we do not differentiate them and call both `heap`.

- free store: managed using `new` and `delete`
- heap: managed using `malloc` and `free`

### Why we need heap

Although constructing objects on stack dispenses a lot of works of deleting a dynamically allocated memory when it is no more needed, sometimes we have no choice but to create objects on heap. For examples,
1. Objects are big. The stack memory is limited, so big objects should not be allocated on stack.
2. The size of objects is uncertain during compiling.
3. The objects are returned by a function but can't be returned by value for some reason.

In those situations, creating objects on stack is not recommended or even impossible. Thus, we still need object construction on heap.

## The dillma of managing objects on heap

Just calling `delete` to free the space on heap seems not a big problem. However, things are not seemingly simple. 

### Dead delete because of exceptions

See the following piece of code

```cpp
void foo()
{
    obj* ptr = new Shape();
    ...
    delete ptr;
}
```

The problem here is that, `delete` may not be called if there is an error thrown before it.

### Easy to forget to delete in complicated function calling

## RAII -- C++-style of memory management

RAII(Resource Acquisition Is Initialization) is a C++-style of memory management, which utilize the stack unwinding to release resources.

### An example of RAII

In [25]:
%%file test.cpp
#include <iostream>

enum class shape_type {
    circle,
    triangle,
    rectangle
};

class shape { 
    public:
        virtual void print() const = 0;
};

class circle: public shape { 
    void print() const override final
    {
        std::cout << "circle\n";
    }
};

class triangle: public shape { 
    void print() const override final
    {
        std::cout << "triangle\n";
    }
};

class rectangle: public shape { 
    void print() const override final
    {
        std::cout << "rectangle\n";
    }
};

shape* create_shape(shape_type type)
{
    shape* ptr = NULL;
    switch(type)
    {
        case shape_type::circle:
            ptr = new circle();
            break;
        case shape_type::triangle:
            ptr = new triangle();
            break;
        case shape_type::rectangle:
            ptr = new rectangle();
            break;
        default:
            throw "No such shape";
    }
    return ptr;
}   

class shape_wrapper
{
    public:
        shape_wrapper(shape* shape_ptr=NULL): shape_ptr_(shape_ptr) {}
        ~shape_wrapper() // release resources
        {
            delete shape_ptr_;
        }
        shape* get() const { return shape_ptr_; }
    private:
        shape* shape_ptr_;
};

int main()
{
    shape_wrapper wrapper(create_shape(shape_type::circle));
    wrapper.get()->print();
    return 0;
}

Overwriting test.cpp


In [26]:
!g++ test.cpp -o test && ./test

circle


In fact, shape_wrapper acts like a smart pointer.

# References
https://diigo.com/0ktz01